希望模型可以基于相同的注意力机制学习到不同的行为， 然后将不同的行为作为知识组合起来， 捕获序列内各种范围的依赖关系 （例如，短距离依赖和长距离依赖关系）。

因此推广到多头注意力.

## 1.模型
给定查询 $q \in \mathbb{R}^{d_q}$、键 $k \in \mathbb{R}^{d_k}$、$v \in \mathbb{R}^{d_v}$,每个注意力头 $h_i$的计算方法为:
$$h_i = f(W_i^{(q)}q,W_i^{(k)}k,W_i^{(v)}v) \in \mathbb{R}^{p_v}$$

其中,$f$可以是加性注意力或缩放点积注意力

In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

## 2.实现
通常选择**缩放点积注意力**作为每一个注意力头

In [2]:
class MultiHeadAttention(nn.Module):
    """多头注意力"""

    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(
            **kwargs)  # 子类可用 super 把关键词参数传回父类的 __init__(). **kwargs表示将所有未命名的关键字参数都作为字典传递给方法。
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        """
        :param queries: 形状为 (batch_size，查询的个数，num_hiddens)
        :param keys:    形状为 (batch_size，“键－值”对的个数，num_hiddens)
        :param values:  形状为 (batch_size，“键－值”对的个数，num_hiddens)
        valid_lens:　   形状为 (batch_size,) 或 (batch_size,查询的个数)

        经过变换后，输出的queries，keys，values　的形状:
        (batch_size * num_heads,查询或者“键－值”对的个数,num_hiddens/num_heads)
        """
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)

        if valid_lens is not None:
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项，然后诸如此类。
            valid_lens = torch.repeat_interleave(valid_lens, repeats=self.num_heads, dim=0)

        # output的形状:(batch_size*num_heads，查询的个数，num_hiddens/num_heads)
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

# 为了实现多头并行计算,转置函数
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，num_hiddens/num_heads)
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)

    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)  # 更改维度的次序

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])


# transpose_output函数反转了transpose_qkv函数的操作
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

一个小例子测试编写的多头注意力类

In [3]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [4]:
batch_size ,num_queries = 2,4 # 一批有 2 份,每份有 4 个元素
num_kvpairs,valid_lens = 6,torch.tensor([3,2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape

torch.Size([2, 4, 100])